In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("secret_tokens")

# folder_path = '/kaggle/working/MMSRec-main'
# remove_folder_contents(folder_path)
# os.rmdir(folder_path)

!git clone https://secret_value_0@github.com/mairyy/MAERec.git

Cloning into 'MAERec'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 74 (delta 15), reused 23 (delta 6), pack-reused 42
Unpacking objects: 100% (74/74), 131.01 MiB | 7.36 MiB/s, done.
Updating files: 100% (37/37), done.


In [3]:
%cd '/kaggle/working/MAERec'

/kaggle/working/MAERec


In [4]:
!pip install "numpy>=1.16.5,<1.23.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 50.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
chex 0.1.81 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.22.4 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.1 which is incompatible.
ydata-profiling 4.3.1 requires scipy<1.11,>=1.4.1, but you have scipy 1.11.1 which is incompatible.


In [5]:
from scipy.sparse import csr_matrix
# from params import args
import numpy as np
import pickle
import copy
import os

def construct_graphs(seq, num_items, distance, prefix):
    user = list()
    r, c, d = list(), list(), list()
    for i, seq in enumerate(seqs):
        print(f"Processing {i}/{len(seqs)} (>﹏<)    ", end='\r')
        for dist in range(1, distance + 1):
            if dist >= len(seq): break;
            r += copy.deepcopy(seq[+dist:])
            c += copy.deepcopy(seq[:-dist])
            r += copy.deepcopy(seq[:-dist])
            c += copy.deepcopy(seq[+dist:])
    d = np.ones_like(r)
    iigraph = csr_matrix((d, (r, c)), shape=(num_items, num_items))
    print('Constructed i-i graph, density=%.6f' % (len(d) / (num_items ** 2)))
    with open(prefix + 'trn', 'wb') as fs:
        pickle.dump(iigraph, fs)

if __name__ == '__main__':

    # dataset = input('Choose a dataset: ')
    dataset = 'clothings'
    prefix = './datasets/' + dataset + '/'

    # distance  = int(input('Max distance of edge: '))
    distance = 3

    with open(prefix + 'seq', 'rb') as fs:
        seqs = pickle.load(fs)

    if dataset == ('books'):
        num_items = 54756
    elif dataset == ('toys'):
        num_items = 54784
    elif dataset == ('retailrocket'):
        num_items = 43886
    elif dataset == ('sports'):
        num_items =  18357
    elif dataset == ('clothings'):
        num_items = 23033
    construct_graphs(seqs, num_items, distance, prefix)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Constructed i-i graph, density=0.002261


In [6]:
import numpy as np
from collections import defaultdict
import pickle

# np.random.seed(12345)

def sample_test_data(data_name, test_num=99, sample_type='random'):
  """
  sample_type:
      random:  sample `test_num` negative items randomly.
      pop: sample `test_num` negative items according to item popularity.
  """

  test_file = './datasets/' + data_name + '/tst'
  sample_file = './datasets/' + data_name + '/neg'
  train_file = './datasets/' + data_name + '/seq'

  item_count = defaultdict(int)
  user_items = defaultdict()
  users = 0

  #read seq in test file
  seqs = []
  with open(test_file, 'rb') as test:
    seqs = pickle.load(test)
  for seq in seqs:
    user_items[users] = seq
    users += 1
    for item in seq:
      item_count[item] += 1

  #read item in train file
  # t_seqs = []
  # with open(train_file, 'rb') as out:
  #   t_seqs = pickle.load(out)
  # for seq in t_seqs:
  #   for item in seq:
  #     item_count[item] += 1

  all_item = list(item_count.keys())
#   print(all_item)
  count = list(item_count.values())
  sum_value = np.sum([x for x in count])
  probability = [value / sum_value for value in count]

  user_neg_items = defaultdict()
#   print(len(all_item))
  for user, user_seq in user_items.items():
    test_samples = []
    # print(user, "length ", len(user_seq))

    while len(test_samples) < test_num:
#       print(len(test_samples))
      if sample_type == 'random':
        sample_ids = np.random.choice(all_item, test_num, replace=False)
        # print("sample: ", sample_ids)
      else: # sample_type == 'pop':
        sample_ids = np.random.choice(all_item, test_num, replace=False, p=probability)

      sample_ids = [item for item in sample_ids if item not in user_seq and item not in test_samples]
      test_samples.extend(sample_ids)
      # print("test: ", test_samples, "length: ", len(test_samples))

    test_samples = test_samples[:test_num]
    user_neg_items[user] = test_samples

  # with open(test_file, 'wb') as out:
  l = []
  for user, samples in user_neg_items.items():
      l.append(samples)
  pickle.dump(l, open(sample_file, 'wb'))

# data_names = ['Beauty', 'Sports_and_Outdoors', 'Toys_and_Games', 'Yelp', 'LastFM']
# for data_name in data_names:
sample_test_data('clothings')

In [7]:
# !sudo apt install python3-scipy
!python main.py --data clothings

Namespace(lr=0.001, seed=19260817, data='clothings', epoch=20, trn_batch=256, tst_batch=256, con_batch=2048, test_frequency=1, max_seq_len=50, num_reco_neg=40, reg=1e-06, ssl_reg=0.01, latdim=32, mask_depth=3, path_prob=0.5, num_attention_heads=4, num_gcn_layers=2, num_trm_layers=2, load_model=None, num_mask_cand=50, mask_steps=10, eps=0.2, attention_probs_dropout_prob=0.3, hidden_dropout_prob=0.3, save_path='tem')
2023-07-26 16:58:07.163892: Start
Traceback (most recent call last):
  File "/kaggle/working/MAERec/main.py", line 319, in <module>
    handler.load_data()
  File "/kaggle/working/MAERec/handler.py", line 68, in load_data
    self.ii_adj = self.make_torch_adj(trn)
  File "/kaggle/working/MAERec/handler.py", line 52, in make_torch_adj
    return t.sparse.FloatTensor(idxs, vals, shape).cuda()
  File "/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py", line 239, in _lazy_init
    raise AssertionError("Torch not compiled with CUDA enabled")
AssertionError: Torch not

In [8]:
import pickle
from torch import nn

def feature(path):
    with open(path, 'rb') as out:
        feature = pickle.load(out)
    return feature

f = feature('./datasets/clothings/tst')
print(len(f))
max_len = 0
for i in f:
    for line in i:
        if line > max_len:
            max_len = line
print(max_len)
print(f[1])


39387
23032
[11, 12, 13, 14]
